In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

In [51]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)

def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1, base_width=64, dilation=1):
        super(Bottleneck, self).__init__()
        width = int(planes * (base_width / 64.)) * groups
        
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out


class ResNet(nn.Module):

    def __init__(self, block, num_classes=10, groups=1, width_per_group=64):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.dilation = 1
        self.groups = groups
        self.base_width = width_per_group
        
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, 3)
        self.layer2 = self._make_layer(block, 128, 4, stride=2)
        self.layer3 = self._make_layer(block, 256, 6, stride=2)
        self.layer4 = self._make_layer(block, 512, 3, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        downsample = None
        previous_dilation = self.dilation
        if self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)

        return x

def ResNet50():
    return ResNet(Bottleneck)

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

EPOCH = 128
BATCH_SIZE = 256
LR = 0.001
 
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4), 
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
 
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
 
trainset = torchvision.datasets.CIFAR10(root='cifar-10-batches-py', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
 
testset = torchvision.datasets.CIFAR10(root='cifar-10-batches-py', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
 
net = ResNet50().to(device)
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=0.0001)

print("Start Training, Resnet-50!")
for epoch in range(EPOCH):
    net.train()
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    data_num = 0
    for i, data in enumerate(trainloader, 0):
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
 
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
 
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        data_num = i
    print('[epoch:%d] Loss: %.03f | Acc: %.3f%% ' 
            % (epoch + 1, sum_loss / (data_num + 1), 100. * correct / total))
    
    #test
    with torch.no_grad():
        correct_test = 0
        total_test = 0
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
        print('Test accuracy: %.3f%%' %(100 * correct_test / total_test))
print("Training Finished, TotalEPOCH=%d" % EPOCH)

Files already downloaded and verified
Files already downloaded and verified
Start Training, Resnet-50!
[epoch:1] Loss: 2.094 | Acc: 24.212% 
Test accuracy: 34.260%
[epoch:2] Loss: 1.725 | Acc: 37.118% 
Test accuracy: 40.700%
[epoch:3] Loss: 1.573 | Acc: 42.962% 
Test accuracy: 45.240%
[epoch:4] Loss: 1.467 | Acc: 46.376% 
Test accuracy: 46.280%
[epoch:5] Loss: 1.394 | Acc: 49.448% 
Test accuracy: 50.110%
[epoch:6] Loss: 1.330 | Acc: 51.822% 
Test accuracy: 51.440%
[epoch:7] Loss: 1.273 | Acc: 54.198% 
Test accuracy: 52.390%
[epoch:8] Loss: 1.229 | Acc: 55.888% 
Test accuracy: 55.500%
[epoch:9] Loss: 1.180 | Acc: 57.798% 
Test accuracy: 55.910%
[epoch:10] Loss: 1.137 | Acc: 59.284% 
Test accuracy: 58.300%
[epoch:11] Loss: 1.103 | Acc: 60.558% 
Test accuracy: 58.590%
[epoch:12] Loss: 1.064 | Acc: 62.052% 
Test accuracy: 59.160%
[epoch:13] Loss: 1.034 | Acc: 63.230% 
Test accuracy: 59.370%
[epoch:14] Loss: 1.006 | Acc: 64.182% 
Test accuracy: 61.400%
[epoch:15] Loss: 0.974 | Acc: 65.292% 

KeyboardInterrupt: 